In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# Carregar dados históricos do IPCA e IGP-M
ipca = pd.read_csv('ipca.csv', index_col=0)
igpm = pd.read_csv('igpm.csv', index_col=0)

# Substituir '--' por NaN antes de converter para valores numéricos
ipca = ipca.replace('--', np.nan)
igpm = igpm.replace('--', np.nan)

# Converter os valores percentuais para decimais
ipca = ipca.applymap(lambda x: float(x.strip('%').replace(',', '.')) / 100 if isinstance(x, str) else x)
igpm = igpm.applymap(lambda x: float(x.strip('%').replace(',', '.')) / 100 if isinstance(x, str) else x)

# Transformar os dados em séries temporais mensais
ipca_series = ipca.stack().reset_index(level=1, drop=True)
igpm_series = igpm.stack().reset_index(level=1, drop=True)

# Ajustar o modelo SARIMAX para o IGP-M
model_igpm = SARIMAX(igpm_series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results_igpm = model_igpm.fit()

# Ajustar o modelo SARIMAX para o IPCA
model_ipca = SARIMAX(ipca_series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results_ipca = model_ipca.fit()

# Fazer previsões para os próximos 180 meses
forecast_igpm = results_igpm.get_forecast(steps=180)
forecast_ipca = results_ipca.get_forecast(steps=180)

# Valores iniciais
valor_inicial = 20000  # R$
salario_mensal = 2000  # R$
aluguel_mensal = 800  # R$
valor_imovel = 200000  # R$
prestacao_financiamento = 600  # R$
meses_financiamento = 180  # 15 anos

# Função para calcular o valor futuro com base nas previsões
def calcular_valor_futuro(valor_inicial, previsoes):
    valor_futuro = valor_inicial
    for i in range(len(previsoes)):
        # Access the predicted value using the iloc accessor
        valor_futuro *= (1 + previsoes.iloc[i])
    return valor_futuro

# Cenário 1: Comprar imóvel financiado
def comprar_imovel(valor_inicial, salario_mensal, prestacao, previsoes_ipca, meses):
    saldo = valor_inicial
    for mes in range(meses):
        saldo += salario_mensal - prestacao
        saldo *= (1 + previsoes_ipca.iloc[mes])
    return saldo

# Cenário 2: Alugar e juntar dinheiro
def alugar_e_juntar(valor_inicial, salario_mensal, aluguel, previsoes_ipca, meses):
    saldo = valor_inicial
    for mes in range(meses):
        saldo += salario_mensal - aluguel
        saldo *= (1 + previsoes_ipca.iloc[mes])
    return saldo

# Calcular os saldos finais para cada cenário
saldo_comprar = comprar_imovel(valor_inicial, salario_mensal, prestacao_financiamento, forecast_ipca.predicted_mean, meses_financiamento)
saldo_alugar = alugar_e_juntar(valor_inicial, salario_mensal, aluguel_mensal, forecast_ipca.predicted_mean, meses_financiamento)

# Calcular o valor futuro do imóvel
valor_imovel_futuro = calcular_valor_futuro(valor_imovel, forecast_igpm.predicted_mean)

# Comparar os resultados
print(f"Saldo final ao comprar o imóvel: R$ {saldo_comprar:.2f}")
print(f"Saldo final ao alugar e juntar dinheiro: R$ {saldo_alugar:.2f}")
print(f"Valor futuro do imóvel: R$ {valor_imovel_futuro:.2f}")

if saldo_comprar + valor_imovel_futuro > saldo_alugar:
    print("Comprar o imóvel é a melhor opção.")
else:
    print("Alugar e juntar dinheiro é a melhor opção.")

<ipython-input-3-eb93864f277b>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ipca = ipca.applymap(lambda x: float(x.strip('%').replace(',', '.')) / 100 if isinstance(x, str) else x)
<ipython-input-3-eb93864f277b>:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  igpm = igpm.applymap(lambda x: float(x.strip('%').replace(',', '.')) / 100 if isinstance(x, str) else x)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored w

Saldo final ao comprar o imóvel: R$ 1230731.20
Saldo final ao alugar e juntar dinheiro: R$ 1083468.84
Valor futuro do imóvel: R$ 20529368.40
Comprar o imóvel é a melhor opção.


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
